In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, classification_report
# from keras.utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dense
import lazypredict
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam

In [23]:
data_path = '011/text_data/'

ada_joy = data_path + 'ada_joy.csv'
assistance_info = data_path + 'assistance_info.csv'
gaze_positions = data_path + 'gaze_positions.csv'
input_info = data_path + 'input_info.csv'
joint_positions = data_path + 'joint_positions.csv'
myo_emg = data_path + 'myo_emg.csv'
myo_imu = data_path + 'myo_imu.csv'
myo_ori = data_path + 'myo_ori.csv'
pupil_cal_eye0 = data_path + 'pupil_cal_eye0.csv'
pupil_cal_eye1 = data_path + 'pupil_cal_eye1.csv'
pupil_eye0 = data_path + 'pupil_eye0.csv'
pupil_eye1 = data_path + 'pupil_eye1.csv'
robot_position = data_path + 'robot_position.csv'
world_cal_positions = data_path + 'world_cal_positions.csv'

ada_joy_df = pd.read_csv(ada_joy)
assistance_info_df = pd.read_csv(assistance_info)
gaze_positions_df = pd.read_csv(gaze_positions)
input_info_df = pd.read_csv(input_info)
joint_positions_df = pd.read_csv(joint_positions)
myo_emg_df = pd.read_csv(myo_emg)
myo_imu_df = pd.read_csv(myo_imu)
myo_ori_df = pd.read_csv(myo_ori)
pupil_cal_eye0_df = pd.read_csv(pupil_cal_eye0)
pupil_cal_eye1_df = pd.read_csv(pupil_cal_eye1)
pupil_eye0_df = pd.read_csv(pupil_eye0)
pupil_eye1_df = pd.read_csv(pupil_eye1)
robot_position_df = pd.read_csv(robot_position)
world_cal_positions_df = pd.read_csv(world_cal_positions)

In [24]:
from sklearn.preprocessing import StandardScaler

# List of all your dataframes
dfs = [ada_joy_df, assistance_info_df, gaze_positions_df, input_info_df, joint_positions_df, myo_emg_df, myo_imu_df, myo_ori_df, pupil_cal_eye0_df, pupil_cal_eye1_df, pupil_eye0_df, pupil_eye1_df, robot_position_df, world_cal_positions_df]

# Data cleaning: Fill missing values with forward fill method
for df in dfs:
    df.fillna(method='ffill', inplace=True)

# Data normalization: Normalize data to have zero mean and unit variance
scaler = StandardScaler()
for df in dfs:
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

C:\Users\test\AppData\Local\Temp\ipykernel_5580\4193131405.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [25]:
print('joint_positions_df columns: ', joint_positions_df.columns)

joint_positions_df columns:  Index(['timestamp', 'world_index', 'world_index_corrected', 'mico_joint_1_pos',
       'mico_joint_2_pos', 'mico_joint_3_pos', 'mico_joint_4_pos',
       'mico_joint_5_pos', 'mico_joint_6_pos', 'mico_joint_finger_1_pos',
       'mico_joint_finger_2_pos', 'mico_joint_1_vel', 'mico_joint_2_vel',
       'mico_joint_3_vel', 'mico_joint_4_vel', 'mico_joint_5_vel',
       'mico_joint_6_vel', 'mico_joint_finger_1_vel',
       'mico_joint_finger_2_vel', 'mico_joint_1_eff', 'mico_joint_2_eff',
       'mico_joint_3_eff', 'mico_joint_4_eff', 'mico_joint_5_eff',
       'mico_joint_6_eff', 'mico_joint_finger_1_eff',
       'mico_joint_finger_2_eff'],
      dtype='object')


In [26]:
joint_cols = ['mico_joint_1_pos', 'mico_joint_2_pos', 'mico_joint_3_pos', 'mico_joint_4_pos', 'mico_joint_5_pos', 'mico_joint_6_pos', 'mico_joint_finger_1_pos', 'mico_joint_finger_2_pos']

for col in joint_cols:
    joint_positions_df[col + '_acceleration'] = joint_positions_df[col].diff().diff()

    joint_positions_df[col + '_jerk'] = joint_positions_df[col].diff().diff().diff()


print('joint_positions_df columns: ', joint_positions_df.shape)


joint_positions_df columns:  (7024, 43)


In [86]:
target_variable = ('mico_joint_1_pos_acceleration', 'mico_joint_1_pos_jerk')

# Identify redundant features and drop them (you may customize this based on your data)
redundant_features = ['world_index_x', 'world_index_corrected_x', 'world_index_y', 'world_index_corrected_y']
redundant_features = [col for col in redundant_features if col in joint_positions_df.columns]  # Check if columns exist
joint_positions_df = joint_positions_df.drop(columns=redundant_features)

# Select features and target variable
features = joint_positions_df.drop(columns=list(target_variable))
target = joint_positions_df[list(target_variable)]

# Check for highly correlated features and drop if necessary
correlation_matrix = features.corr()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_triangle.columns if any(np.abs(upper_triangle[column]) > 0.95)]
features = features.drop(columns=to_drop)

# Confirm the selected features and target variable
print("Selected Features:", features.shape)
print("Target Variable:", target_variable)

Selected Features: (7024, 39)
Target Variable: ('mico_joint_1_pos_acceleration', 'mico_joint_1_pos_jerk')


In [94]:
# this are selected features: joint_positions_df columns:  (7024, 43)
# Selected Features: (7024, 39)
# Target Variable: ('mico_joint_1_pos_acceleration', 'mico_joint_1_pos_jerk') 

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)
# Normalize data
# scaler = MinMaxScaler(feature_range=(0, 1))
# X_train_scaled = scaler.fit_transform(X_train)


# X_train = np.array(X_train)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

# X_train.astype

In [95]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)
models


  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [00:00<00:00, 51.35it/s]

Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


In [96]:
# Reshape data for LSTM model
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
# X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
# '''
# Model building
num_features = X_train.shape[2]
num_outputs = len(y_train.columns)

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(LSTM(50))
model.add(Dense(num_outputs))

model.compile(loss='mean_squared_error', optimizer='adam')

# Model training
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_split=0.2)
# '''


Epoch 1/100
141/141 [==============================] - 5s 13ms/step - loss: nan - val_loss: nan
Epoch 2/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 3/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 4/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 5/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 6/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 7/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 8/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 9/100
141/141 [==============================] - 1s 9ms/step - loss: nan - val_loss: nan
Epoch 10/100
141/141 [==============================] - 1s 8ms/step - loss: nan - val_loss: nan
Epoch 11/100
141/141 [==========================

KeyboardInterrupt: 

In [71]:
# from sklearn.model_selection import train_test_split
# import numpy as np

# # Assuming 'target' is the column you want to predict
# X = joint_positions_df.drop(list(target_variable), axis=1)
# y = joint_positions_df[list(target_variable)]

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# # Reshape input to be 3D [samples, timesteps, features]
# # X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
# # X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# X_train.shape, joint_positions_df.shape

((5619, 41), (7024, 43))